## Import Statement

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nnsplit import NNSplit
import pandas as pd 
import re

## Load data and get all the reviews

In [36]:
df = pd.read_csv("./data/RGeo_Hotel_Reviews.csv", index_col=0)
df

,Additional_Number_of_Scoring,Average_Score,Hotel_Address,Hotel_Name,Negative_Review,Positive_Review,Review_Date,Review_Total_Negative_Word_Counts,Review_Total_Positive_Word_Counts,Reviewer_Nationality,Reviewer_Score,Tags,Total_Number_of_Reviews,Total_Number_of_Reviews_Reviewer_Has_Given,city,country,days_since_review,lat,lng
0,194,7.7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,8/3/2017,397,11,Russia,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",1403,7,Amsterdam,Netherlands,0 days,52.360576,4.915968
1,194,7.7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Hotel Arena,No Negative,No real complaints the hotel was great great ...,8/3/2017,0,105,Ireland,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",1403,7,Amsterdam,Netherlands,0 days,52.360576,4.915968
2,194,7.7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7/31/2017,42,21,Australia,7.1,"[' Leisure trip ', ' Family with young childre...",1403,9,Amsterdam,Netherlands,3 days,52.360576,4.915968
3,194,7.7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,7/31/2017,210,26,United Kingdom,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",1403,1,Amsterdam,Netherlands,3 days,52.360576,4.915968
4,194,7.7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting,7/24/2017,140,8,New Zealand,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",1403,3,Amsterdam,Netherlands,10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512465,168,8.1,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,Atlantis Hotel Vienna,no trolly or staff to help you take the lugga...,location,8/30/2015,14,2,Kuwait,7.0,"[' Leisure trip ', ' Family with older childre...",2823,8,Vienna,Austria,704 day,48.203745,16.335677
512466,168,8.1,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,Atlantis Hotel Vienna,The hotel looks like 3 but surely not 4,Breakfast was ok and we got earlier check in,8/22/2015,11,11,Estonia,5.8,"[' Leisure trip ', ' Family with young childre...",2823,12,Vienna,Austria,712 day,48.203745,16.335677
512467,168,8.1,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,Atlantis Hotel Vienna,The ac was useless It was a hot week in vienn...,No Positive,8/19/2015,19,0,Egypt,2.5,"[' Leisure trip ', ' Family with older childre...",2823,3,Vienna,Austria,715 day,48.203745,16.335677
512468,168,8.1,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,Atlantis Hotel Vienna,No Negative,The rooms are enormous and really comfortable...,8/17/2015,0,25,Mexico,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",2823,3,Vienna,Austria,717 day,48.203745,16.335677


## Filter the columns and combine positive and negative review

In [197]:
df_negative_review = df[['Negative_Review', 'city', 'country']]
df_positive_review = df[['Positive_Review', 'city', 'country']]
df_uncleaned_reviews = pd.concat([df_negative_review, df_positive_review], ignore_index=True)

/Users/yuankanglee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [198]:
df_uncleaned_reviews

,Negative_Review,Positive_Review,city,country
0,I am so angry that i made this post available...,NaN,Amsterdam,Netherlands
1,No Negative,NaN,Amsterdam,Netherlands
2,Rooms are nice but for elderly a bit difficul...,NaN,Amsterdam,Netherlands
3,My room was dirty and I was afraid to walk ba...,NaN,Amsterdam,Netherlands
4,You When I booked with your company on line y...,NaN,Amsterdam,Netherlands
...,...,...,...,...
1024935,NaN,location,Vienna,Austria
1024936,NaN,Breakfast was ok and we got earlier check in,Vienna,Austria
1024937,NaN,No Positive,Vienna,Austria
1024938,NaN,The rooms are enormous and really comfortable...,Vienna,Austria


In [208]:
df_uncleaned_reviews['Review'] = df_uncleaned_reviews['Negative_Review'].combine_first(df_uncleaned_reviews['Positive_Review'])
df_reviews = df_uncleaned_reviews.drop(columns=['Negative_Review', 'Positive_Review'])

In [209]:
df_reviews

,city,country,Review
0,Amsterdam,Netherlands,I am so angry that i made this post available...
1,Amsterdam,Netherlands,No Negative
2,Amsterdam,Netherlands,Rooms are nice but for elderly a bit difficul...
3,Amsterdam,Netherlands,My room was dirty and I was afraid to walk ba...
4,Amsterdam,Netherlands,You When I booked with your company on line y...
...,...,...,...
1024935,Vienna,Austria,location
1024936,Vienna,Austria,Breakfast was ok and we got earlier check in
1024937,Vienna,Austria,No Positive
1024938,Vienna,Austria,The rooms are enormous and really comfortable...


## Splitting reviews in to sentence using NNSplit

In [65]:
splitter = NNSplit("en")
df_reviews['Review'].apply(lambda x: [splitter.split([word]) for word in x]  )

KeyboardInterrupt: 

In [210]:
df_reviews_test = df_reviews[5:8]

In [211]:
df_reviews_test

,city,country,Review
5,Amsterdam,Netherlands,Backyard of the hotel is total mess shouldn t...
6,Amsterdam,Netherlands,Cleaner did not change our sheet and duvet ev...
7,Amsterdam,Netherlands,Apart from the price for the brekfast Everyth...


In [212]:
splitter = NNSplit("en")
df_reviews_test['Review'] = df_reviews_test['Review'].apply(lambda x: splitter.split([x.strip()]))

/Users/yuankanglee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [213]:
df_reviews_test['Review']

5    [[[Token(text='Backyard', whitespace=' '), Tok...
6    [[[Token(text='Cleaner', whitespace=' '), Toke...
7    [[[Token(text='Apart', whitespace=' '), Token(...
Name: Review, dtype: object

In [214]:
for i in df_reviews_test['Review']:
    new_string = ''
    for j in i:
        for k in j:
            for l in k:
                new_string += l.text + " "
    print(new_string)

Backyard of the hotel is total mess shouldn t happen in hotel with 4 stars 
Cleaner did not change our sheet and duvet everyday but just made bed They also didn t clean the floor and changed the body gel when we run out of it 
Apart from the price for the brekfast Everything very good 


In [215]:
def make_sent(x):
    print(x)
    new_string = ''
    for j in x:
        for k in j:
            for l in k:
                new_string += l.text + " "
    print(new_string)
    print("---------")
    return new_string

df_reviews_test['Review'] = df_reviews_test['Review'].apply(lambda x: make_sent(x))

[[[Token(text='Backyard', whitespace=' '), Token(text='of', whitespace=' '), Token(text='the', whitespace=' '), Token(text='hotel', whitespace=' '), Token(text='is', whitespace=' '), Token(text='total', whitespace=' '), Token(text='mess', whitespace=' '), Token(text='shouldn', whitespace=' '), Token(text='t', whitespace=' '), Token(text='happen', whitespace=' '), Token(text='in', whitespace=' '), Token(text='hotel', whitespace=' '), Token(text='with', whitespace=' '), Token(text='4', whitespace=' '), Token(text='stars', whitespace='')]]]
Backyard of the hotel is total mess shouldn t happen in hotel with 4 stars 
---------
[[[Token(text='Cleaner', whitespace=' '), Token(text='did', whitespace=' '), Token(text='not', whitespace=' '), Token(text='change', whitespace=' '), Token(text='our', whitespace=' '), Token(text='sheet', whitespace=' '), Token(text='and', whitespace=' '), Token(text='duvet', whitespace=' '), Token(text='everyday', whitespace=' '), Token(text='but', whitespace=' '), T

/Users/yuankanglee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [216]:
df_reviews_test

,city,country,Review
5,Amsterdam,Netherlands,Backyard of the hotel is total mess shouldn t ...
6,Amsterdam,Netherlands,Cleaner did not change our sheet and duvet eve...
7,Amsterdam,Netherlands,Apart from the price for the brekfast Everythi...
